In [1]:
# Importing required packages

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass
import html2text as h2h
from html.parser import HTMLParser

#required packages
import mysql.connector as connection
from functools import reduce


In [2]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

## Final Function
def f_final_function(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_host_input = input("Enter the host: ")
            v_user_input = input("Enter the username: ")
            v_password_input = getpass.getpass("Enter the password: ")
            v_database_input = input("Enter the database: ")
            v_mydb,v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

            if v_mydb is not None:

                #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break

        # Query 1
        v_queries = {'Query1':'''SELECT v.visit_id AS Visit_id,
                    v.patient_id AS Patient_id,
                    MAX(pi.identifier) AS OpenMRS_id,
                    MAX(CONCAT(pn.given_name, ' ', pn.family_name)) as Patient_name,
                    p.gender AS Gender,
                    p.birthdate AS Date_of_birth,
                    YEAR(CURDATE()) - YEAR(p.birthdate) AS Age,
                    u.username as CHW_name,
                    v.date_Started AS Visit_started_date,
                    v.date_stopped AS Visit_ended_date,
                    v.date_created AS Visit_creation_date,
                    v.creator AS Creator_Id,
                    l.city_village as Location_village,
                    ls.name AS Location_sanch,
                    ld.name AS Location_district,
                    l.state_province as Location_state,
                    la.value_reference as Village_Type
                FROM visit v
                JOIN person p ON v.patient_id = p.person_id
                JOIN person_name pn ON v.patient_id=pn.person_id
                LEFT JOIN location l ON v.location_id=l.location_id
                JOIN users u ON u.user_id=v.creator
                JOIN patient_identifier pi ON pi.patient_id=v.patient_id
                LEFT JOIN location ls on (ls.location_id = l.parent_location)
                LEFT JOIN location ld on (ld.location_id = ls.parent_location )
                LEFT JOIN location lst on (lst.location_id = ld.parent_location )
                LEFT JOIN location_attribute la on (la.voided = 0 and la.attribute_type_id = 11 and la.location_id = l.location_id)
                WHERE v.voided = 0
                AND p.voided=0
                AND pn.preferred = 1
                AND pn.voided=0
                AND pi.identifier_type=3
                AND  pi.voided=0
                AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id, v.date_Started, v.date_created,
                     v.date_stopped, v.creator, v.patient_id,
                      l.state_province, l.city_village,
                    CASE WHEN l.parent_location IS NOT NULL THEN  l.name ELSE l.city_village END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  ls.name ELSE NULL END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  ld.name ELSE l.county_district END ,
                    CASE WHEN l.parent_location IS NOT NULL THEN  lst.name ELSE l.state_province END,
                    la.value_reference;''',

        'Query2':'''SELECT v.visit_id as Visit_id,
                    e.creator AS Doctor_id
                    FROM visit v
                    JOIN encounter e ON e.visit_id=v.visit_id 
                    WHERE v.voided=0
                          AND e.encounter_type=9
                          AND e.voided=0
                          AND v.date_Started BETWEEN %s AND %s;''',

        'Query3':'''SELECT v.visit_id as Visit_id,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163212 THEN o.value_text ELSE NULL END) AS Chief_complaint,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163210 THEN o.value_text ELSE NULL END) AS Patient_medical_history,
                    GROUP_CONCAT(distinct CASE WHEN o.concept_id = 163211 THEN o.value_text ELSE NULL END) AS Family_history
                FROM visit v
                JOIN encounter e ON v.visit_id = e.visit_id
                JOIN obs o ON e.encounter_id = o.encounter_id
                WHERE v.voided = 0 
                    AND e.encounter_type = 1
                    AND e.voided = 0
                    AND o.voided = 0
                    AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id
                ORDER BY v.visit_id;''',

        'Query4':'''SELECT v.visit_id as Visit_id,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163219 THEN o.value_text ELSE NULL END) AS Diagnosis,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163202 THEN o.value_text ELSE NULL END SEPARATOR ';') AS Medications,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163206 THEN o.value_text ELSE NULL END) AS Medical_test,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163205 THEN o.value_text ELSE NULL END) AS Medical_advice,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 165095 THEN o.value_text ELSE NULL END) AS Notes,
                       GROUP_CONCAT(CASE WHEN o.concept_id = 163345 THEN o.value_text ELSE NULL END) AS Follow_up_date
                FROM visit v
                JOIN encounter e ON e.visit_id=v.visit_id
                JOIN obs o ON o.encounter_id=e.encounter_id
                WHERE v.voided=0
                      AND e.encounter_type=9
                      AND e.voided=0
                      AND o.voided=0
                      AND v.date_Started BETWEEN %s AND %s
                GROUP BY v.visit_id
                ORDER BY v.visit_id;''',

        'Query5':'''SELECT v.visit_id as Visit_id,
               MAX(CASE WHEN va.attribute_type_id = 5 THEN va.value_reference END) AS Specialty,
                MAX(CASE WHEN va.attribute_type_id = 7 THEN va.value_reference END) AS Visit_upload_time
                FROM visit v
                JOIN visit_attribute va ON va.visit_id=v.visit_id
                WHERE v.voided=0
                      AND (va.attribute_type_id=5 OR va.attribute_type_id=7)
                      AND va.voided=0
                      AND v.date_Started BETWEEN %s AND %s
               GROUP BY v.visit_id
            ORDER BY v.visit_id;''',


        'Query6':'''SELECT v.visit_id as Visit_id,
                       MAX(distinct CASE WHEN e.encounter_type = 9 THEN e.encounter_datetime ELSE NULL END) AS Consultation_start_time,
                       MAX(CASE WHEN e.encounter_type = 14 THEN CONCAT(pn.given_name, ' ', pn.family_name) END) AS Doctor_name,
                       TIME_TO_SEC(SEC_TO_TIME(HOUR(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created))*3600 +
                                  (MINUTE(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created))*60) +
                                  (SECOND(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime
                                  ELSE NULL END),v.date_created)))))/ 60 AS Doctor_TAT,
                       TIME_TO_SEC(SEC_TO_TIME(HOUR(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started))*3600 +
                                  (MINUTE(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started))*60)+
                                  (SECOND(TIMEDIFF(MAX(CASE WHEN e.encounter_type = 14 THEN v.date_stopped
                                  ELSE e.encounter_datetime END), v.date_started)))))/ 60 AS Patient_TAT,
              MAX(distinct CASE WHEN e.encounter_type = 14 THEN e.encounter_datetime ELSE NULL END) AS Sign_submit_time
                FROM visit v
                JOIN encounter e ON e.visit_id=v.visit_id
                JOIN users u ON u.user_id=e.creator
                JOIN person_name pn ON pn.person_id=u.person_id
                WHERE v.voided=0
                      AND e.voided=0
                      AND pn.voided=0
                      AND v.date_Started BETWEEN %s AND %s
                GROUP by v.visit_id;'''}

        # Here append all the quries
        #v_query2 = [q1,q2,q3,q4,q5,q6,q7]


        def f_get_date_range_from_tables(p_mydb, p_cursor):
            try:
                if p_mydb.is_connected():
                    # SQL query to fetch the minimum and maximum dates from "visit" table
                    date_query = "SELECT MIN(date_Started) AS min_date, MAX(date_Started) AS max_date FROM visit WHERE voided = 0;"

                    p_cursor.execute(date_query)
                    v_date_results = p_cursor.fetchall()

                    if v_date_results:
                        v_min_date, v_max_date = v_date_results[0]

                        return v_min_date, v_max_date

                    else:
                        print("No data available in the tables.")
                        return None, None
                else:
                    print("Database connection is not opened.")
                    return None, None
            except Exception as e:
                print(f"An error occurred: {e}")


        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        #print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")

        # Function to execute a query with date input parameters
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)

                    return df
                else:
                    print("Database connection is not opened.")
                    return None
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
            
        #Creating a function that will extract the data before the associated symptoms
        def f_extract_section(p_text):
            try:
                v_matches = re.findall(r'(.*?)Associated symptoms', p_text, re.DOTALL)
                if v_matches:
                    return v_matches[0].strip()
                else:
                    return p_text.strip()
            except Exception as e:
                print(f"Error: {e}")
                return p_text.strip()

        # Extracting the Chief complaint from the data
        def f_extract_all_symptoms(p_text):
            try:
                v_matches = re.findall(r'►<b>(.*?)</b>', p_text)
                if v_matches:
                    v_formatted_symptoms = ', '.join(v_matches)
                    return v_formatted_symptoms
            except:
                print("An error occurred while extracting symptoms.")
            return None


        #Combining both the functions to extract the all the chief complaints
        def f_combined_extraction(p_text):
            v_section = f_extract_section(p_text)
            if v_section:
                return f_extract_all_symptoms(v_section)
            return None


        ## Function for segregation, we can simply pass the dataframe and get the output
        def f_data_segregation(v_sample):
            try:
                merged_df = v_sample

                # Getting the required columns in order for the new one
                v_df_new = merged_df[['Visit_id','Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village','Village_Type','Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Chief_complaint','Consultation_start_time','Doctor_name','Doctor_TAT','Patient_TAT','Diagnosis','Medications','Medical_test','Medical_advice','Notes','Follow_up_date','Patient_medical_history','Family_history','Sign_submit_time', 'Patient_waiting_time_(in_hours)']]

                ## Here getting the remaining columns
                v_df_additional = merged_df.drop(['Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village','Village_Type','Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Chief_complaint','Consultation_start_time','Doctor_name','Doctor_TAT','Patient_TAT','Diagnosis','Medications','Medical_test','Medical_advice','Notes','Follow_up_date','Patient_medical_history','Family_history','Sign_submit_time', 'Patient_waiting_time_(in_hours)'], axis=1)

                ## Here merged the two dataframe
                v_df_merged = pd.merge(v_df_new, v_df_additional, on='Visit_id')

                # copy the datafrmae
                df_cc=v_df_merged.copy()

                # selecting the required columns for 'chief complaint' to make it as a dataframe
                df_cc = df_cc[['Visit_id','Patient_id','Visit_started_date','Follow_up_date', 'Chief_complaint']]

                #Imputing the nan values in the chief complaint data with null
                df_cc['Chief_complaint'] = df_cc['Chief_complaint'].fillna('null')

                #Removing the unwanted symbols in the chief complaint data
                df_cc['Chief_complaint'] = df_cc['Chief_complaint'].replace('\xa0', ' ', regex=True)


                #Applying the function to the dataframe
                df_cc['Symptoms'] = df_cc['Chief_complaint'].apply(f_combined_extraction)

                #remove duplicates
                df_cc = df_cc.drop_duplicates(['Visit_id','Patient_id', 'Visit_started_date'])

                # Merge final_sql_df and final_df using 'inner' join
                complete_df = pd.merge(v_df_merged, df_cc, on=['Visit_id', 'Patient_id', 'Visit_started_date','Chief_complaint','Follow_up_date'], how='left')

                # Drop the duplicates
                complete_df = complete_df.drop_duplicates()

                # Rearrange the columns
                v_last_df = complete_df[['Visit_id', 'Patient_id','OpenMRS_id','Gender','Date_of_birth','Age','Patient_name','Location_state','Location_district','Location_sanch','Location_village','Village_Type',
                                         'Visit_started_date','Visit_creation_date','Visit_ended_date','CHW_name','Specialty','Visit_upload_time','Symptoms','Chief_complaint','Consultation_start_time','Doctor_name','Doctor_TAT',
                                        'Patient_TAT','Diagnosis','Medications', 'Medical_test','Medical_advice','Notes','Follow_up_date','Patient_medical_history','Family_history','Sign_submit_time','Patient_waiting_time_(in_hours)','Creator_Id','Doctor_id']].copy()

                # Export the dataframe to excel
                #v_last_df.to_excel('Final_df1.xlsx',index=False)
                print('Segregated columns added\n')

                # Get the current date and time
                file_name = datetime.now()
                # Format the datetime object
                formatted_file_name = int(file_name.strftime("%Y%m%d%H%M%S"))

                # It'll create the filename with date and time
                v_last_df.to_excel(f'PWT_with_Clinical_notes_{formatted_file_name}.xlsx', index=False)

                print(f'Final "PWT_with_Clinical_notes_{formatted_file_name}.xlsx" excel output created...:)\n')
                # It returns the final dataframe
                return v_last_df

            except Exception as e:
               print('Error in the data segregation function', e)


        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second

        print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')

        try:
            while True:
                # Getting input from user
                v_strt_date_input = v_visit_started_date

                pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                #v_start_date = v_strt_date_input+' '+str(v_hours)+':'+str(v_minutes)+':'+str(v_seconds)
                v_start_date = v_strt_date_input+' 00:00:00'
                date_format = '%Y-%m-%d %H:%M:%S'

                # Convert the string to a datetime object
                datetime_object = datetime.strptime(v_start_date, date_format)
                print("\nDate Entered: \t",datetime_object)

                try:
                    if (datetime_object<=v_last_date): # (datetime_object>=v_min_date) and 
                        while True:
                            # Getting input from user
                            v_end_date_input = v_visit_ended_date
                            pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                            v_end_date = v_end_date_input+' 23:59:59'

                            #v_end_date = v_end_date+' 00:00:00'
                            # Convert the string to a datetime object
                            datetime_object1 = datetime.strptime(v_end_date, date_format)
                            print("\nDate Entered: \t",datetime_object1,'\n')

                            try:
                                if (datetime_object1<v_last_date) or (datetime_object1>datetime_object):
                                    if datetime_object1 < datetime_object:
                                        print('Incorrct Date. Please Enter the Date within the range.')
                                    else:
                                        # Getting the username
                                        v_hostname = socket.gethostname()
                                        v_host_name.append(v_hostname)

                                        # Entered data ranges
                                        date_range.append(v_start_date+' / '+v_end_date)
                                        
                                        # Passing the startdate, end date and query in the below function to fetch the data
                                        #v_sample = f_filter_data(datetime_object, datetime_object1, v_queries)
                                        
                                        # Execute each query with input parameters
                                        v_params = (v_start_date, v_end_date)
                                        
                                        #connect database and cursor     
                                        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)
                                            
                                        # Execute each query with input parameters and store the output in a dictionary
                                        v_output_dataframes = {}
                                        for query_name, query in v_queries.items():
                                            
                                            df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                                            
                                            if df is not None:
                                                v_output_dataframes[query_name] = df
                                                print(f"{query_name} executed successfully.\n")

                                        # Close the database connection
                                        f_close_database_connection(v_mydb, v_cursor)

                                        # Merge all the data frames based on 'visit_id' using 'outer' join
                                        merged_df = None
                                        for df in v_output_dataframes.values():
                                            if merged_df is None:
                                                merged_df = df
                                            else:
                                                merged_df = pd.merge(merged_df, df, on='Visit_id', how='outer')
                                        
                                        v_sample = merged_df.copy()
                                        print('\nCombined dataframe created\n')
                                        
                                        # After fetching the data need to calculate the new column Patient_waiting_time_(in_hours)
                                        
                                        # Convert the 'Visit_started_date' and 'Sign_submit_time' columns to datetime objects
                                        v_sample['Visit_started_date'] = pd.to_datetime(v_sample['Visit_started_date'])
                                        v_sample['Sign_submit_time'] = pd.to_datetime(v_sample['Sign_submit_time'])


                                        # Calculate the time difference in hours
                                        print('Patient_waiting_time_(in_hours) column added\n')
                                        v_sample['Patient_waiting_time_(in_hours)'] = (v_sample['Sign_submit_time'] - v_sample['Visit_started_date']).dt.total_seconds() / 3600
                                        #v_sample.to_excel('Demo2.xlsx', index=False)
                                        
                                        # After getting Patient_waiting_time_(in_hours) this column pass the dataframe to segregation
                                        df_segregate = f_data_segregation(v_sample)
                                        
                                        end = datetime.now()
                                        ed.append(end)

                                        duration = ed[-1]-st[-1]
                                        du.append(duration)

                                        dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                                        duration_df = pd.DataFrame(dur)

                                        # Convert the 'Duration' column to seconds
                                        duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                                        duration_df.to_excel('Duration.xlsx', index=False)
                                        
                                        #print('Executed successfully ')
                                        
                                        # It returns the segregated dataframe and the duration dataframe
                                        
                                        return df_segregate, duration_df
                                        
                                else:
                                    print('Enter the date with range')
                            except Exception as e:
                                print('Error 2', e)
                                break
                        break
                    else:
                        print('Error3')
                    #break
                except Exception as e:
                            print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)

In [3]:
v_data_seg = f_final_function('2024-01-01', '2024-03-31')

Enter the host: afitraining.ekalarogya.org
Enter the username: root
Enter the password: ········
Enter the database: openmrs
Connected to the database

Cursor closed.
Database connection closed.

Connected to the database

Cursor closed.
Database connection closed.

Date Between **Starting Date - (2023-05-12 13:48:58) and **End Date - (2024-04-25 14:45:25).

**The format is yyy-mm-dd


Date Entered: 	 2024-01-01 00:00:00

Date Entered: 	 2024-03-31 23:59:59 

Connected to the database

Query1 executed successfully.

Query2 executed successfully.

Query3 executed successfully.

Query4 executed successfully.

Query5 executed successfully.

Query6 executed successfully.

Cursor closed.
Database connection closed.


Combined dataframe created

Patient_waiting_time_(in_hours) column added

Segregated columns added

Final "PWT_with_Clinical_notes_20240425150715.xlsx" excel output created...:)



In [4]:
v_df = v_data_seg[0]
v_df

,Visit_id,Patient_id,OpenMRS_id,Gender,Date_of_birth,Age,Patient_name,Location_state,Location_district,Location_sanch,Location_village,Village_Type,Visit_started_date,Visit_creation_date,Visit_ended_date,CHW_name,Specialty,Visit_upload_time,Symptoms,Chief_complaint,Consultation_start_time,Doctor_name,Doctor_TAT,Patient_TAT,Diagnosis,Medications,Medical_test,Medical_advice,Notes,Follow_up_date,Patient_medical_history,Family_history,Sign_submit_time,Patient_waiting_time_(in_hours),Creator_Id,Doctor_id
0,2587,492736,2C62M-9,F,1964-01-02,60,Sudama Devi,jharkhand,Deoghar,Dumma,Udaypura,Primary,2024-01-02 13:22:09,2024-01-02 13:37:31,NaT,nurse1,General Physician,02 Jan 2024 13:37:24,"Shoulder, arm or hand pain","►<b>Shoulder, arm or hand pain</b>: <br/>• Sit...",None,None,None,6.3333,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
1,2588,492738,2C62P-2,M,1991-01-02,33,Shankar Test,Jharkhand,Giridih,Parasnath,Chatro,None,2024-01-02 19:32:31,2024-01-02 19:43:58,NaT,nurse,General Physician,02 Jan 2024 19:43:53,Fever,►<b>Fever</b>: <br/>• Duration - 2 Days.<br/>•...,None,None,None,5.8500,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,NaT,NaN,911,NaN
2,2589,492761,2C63G-1,F,1995-08-21,29,Test VillageA,Uttar Pradesh,Sonbhadra,Myorpur,Aajangira,None,2024-01-05 11:50:26,2024-01-05 11:56:43,NaT,nurse1,General Physician,05 Jan 2024 11:56:39,Blood in stool or Black stool,►<b>Blood in stool or Black stool</b>: <br/>• ...,None,None,None,5.2000,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Patient is unmarried.<br...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
3,2590,492762,2C63H-9,F,1992-09-01,32,Test VillageB,Uttar Pradesh,Sonbhadra,Myorpur,Aarangpani,None,2024-01-05 11:54:29,2024-01-05 12:00:51,NaT,nurse1,General Physician,05 Jan 2024 12:00:47,Burns,►<b>Burns</b>: <br/>• Location - test.<br/>• 5...,None,None,None,5.4667,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Not pregnant.<br/>• Medi...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
4,2591,492763,2C63J-4,F,1997-09-01,27,Test VillageC,Uttar Pradesh,Sonbhadra,Myorpur,Aarangpani,None,2024-01-05 12:00:44,2024-01-05 12:07:01,NaT,nurse1,General Physician,05 Jan 2024 12:06:56,Abdominal Pain,►<b>Abdominal Pain</b>: <br/>• Site - Upper (R...,None,None,None,5.2667,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Patient is unmarried.<br...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2861,493214,2C716-8,M,1996-03-29,28,Test March Prescription,Maharashtra,Nashik,Jogmudi,Anbe,None,2024-03-29 11:12:18,2024-03-29 11:18:41,2024-04-01 11:02:57,nurseqa,General Physician,29 Mar 2024 11:18:35,Burns,►<b>Burns</b>: <br/>• Location - hand.<br/>• 1...,2024-03-29 11:18:57,doctorqa qa,1.2333,4310.6500,Oedema:Secondary & Confirmed,"Ibuprofen Tablets 400mg: 400 mg, 6 Tablet Once...","CBC: HB/WBC/RBC PLATELET,UREA AND CREATININE TEST","<a href=""tel:9006969668"">Start Audio Call with...",None,"29-03-2024, Remark: 2nd prescription",• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,2024-03-29 11:19:55,0.126944,1058,1059.0
275,2862,493215,2C717-6,F,1990-03-29,34,Test Eeshani Charama,Maharashtra,Nashik,Jogmudi,Anbe,None,2024-03-29 11:29:40,2024-03-29 11:36:01,2024-03-29 11:38:51,nurseqa,General Physician,29 Mar 2024 11:35:56,Ear pain,►<b>Ear pain</b>: <br/>• Affected ear(s): - Le...,2024-03-29 11:35:46,doctorqa qa,0.8833,9.1833,"SEIZURE:Secondary & Confirmed,Malaria sevère:P...","Paracetamol Tablets 500mg: 500 mg, 6 Tablet On...","CBC: HB/WBC/RBC PLATELET,STOOL: OCCULT BLOOD A...","<a href=""tel:9006969668"">Start Audio Call with...",None,"29-03-2024, Remark: abc",• Pregnancy status* - Not pregnan

In [5]:
duration_df = v_data_seg[1]

In [8]:
v_df.head(15)

,Visit_id,Patient_id,OpenMRS_id,Gender,Date_of_birth,Age,Patient_name,Location_state,Location_district,Location_sanch,Location_village,Village_Type,Visit_started_date,Visit_creation_date,Visit_ended_date,CHW_name,Specialty,Visit_upload_time,Symptoms,Chief_complaint,Consultation_start_time,Doctor_name,Doctor_TAT,Patient_TAT,Diagnosis,Medications,Medical_test,Medical_advice,Notes,Follow_up_date,Patient_medical_history,Family_history,Sign_submit_time,Patient_waiting_time_(in_hours),Creator_Id,Doctor_id
0,2587,492736,2C62M-9,F,1964-01-02,60,Sudama Devi,jharkhand,Deoghar,Dumma,Udaypura,Primary,2024-01-02 13:22:09,2024-01-02 13:37:31,NaT,nurse1,General Physician,02 Jan 2024 13:37:24,"Shoulder, arm or hand pain","►<b>Shoulder, arm or hand pain</b>: <br/>• Sit...",None,None,None,6.3333,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
1,2588,492738,2C62P-2,M,1991-01-02,33,Shankar Test,Jharkhand,Giridih,Parasnath,Chatro,None,2024-01-02 19:32:31,2024-01-02 19:43:58,NaT,nurse,General Physician,02 Jan 2024 19:43:53,Fever,►<b>Fever</b>: <br/>• Duration - 2 Days.<br/>•...,None,None,None,5.8500,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo...,NaT,NaN,911,NaN
2,2589,492761,2C63G-1,F,1995-08-21,29,Test VillageA,Uttar Pradesh,Sonbhadra,Myorpur,Aajangira,None,2024-01-05 11:50:26,2024-01-05 11:56:43,NaT,nurse1,General Physician,05 Jan 2024 11:56:39,Blood in stool or Black stool,►<b>Blood in stool or Black stool</b>: <br/>• ...,None,None,None,5.2000,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Patient is unmarried.<br...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
3,2590,492762,2C63H-9,F,1992-09-01,32,Test VillageB,Uttar Pradesh,Sonbhadra,Myorpur,Aarangpani,None,2024-01-05 11:54:29,2024-01-05 12:00:51,NaT,nurse1,General Physician,05 Jan 2024 12:00:47,Burns,►<b>Burns</b>: <br/>• Location - test.<br/>• 5...,None,None,None,5.4667,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Not pregnant.<br/>• Medi...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
4,2591,492763,2C63J-4,F,1997-09-01,27,Test VillageC,Uttar Pradesh,Sonbhadra,Myorpur,Aarangpani,None,2024-01-05 12:00:44,2024-01-05 12:07:01,NaT,nurse1,General Physician,05 Jan 2024 12:06:56,Abdominal Pain,►<b>Abdominal Pain</b>: <br/>• Site - Upper (R...,None,None,None,5.2667,NaN,NaN,NaN,NaN,NaN,NaN,• Pregnancy status* - Patient is unmarried.<br...,•Do you have a family history of any of the fo...,NaT,NaN,318,NaN
5,2592,492764,2C63K-2,M,1991-01-05,33,Test Anbi,Maharashtra,Nashik,Jogmudi,Anbe,None,2024-01-05 12:43:53,2024-01-05 12:49:57,2024-02-16 11:56:21,nurseqa,General Physician,05 Jan 2024 12:49:52,Cough,►<b>Cough</b>: <br/>• Duration - Acute (0-2 we...,2024-01-16 10:05:17,doctorqa qa,15681.3833,50339.9833,"Rabies:Secondary & Confirmed,Asthma:Secondary ...",None,UREA AND CREATININE TEST,"<a href=""tel:9006969668"">Start Audio Call with...",None,"18-01-2024, Remark: check authantication",• Medical History - Asthma - Current medicatio...,•Do you have a family history of any of the fo...,2024-01-16 10:11:20,261.457500,1058,1059.0
6,2593,492765,2C63L-0,M,1992-01-05,32,Test Badgi Second,Maharashtra,Nashik,Jogmudi,Badgi,None,2024-01-05 12:48:04,2024-01-05 12:54:03,NaT,nurseqa,General Physician,05 Jan 2024 12:53:58,Burns,►<b>Burns</b>: <br/>• Location - wj.<br/>• 1 H...,None,None,None,5.1000,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - Tuberculosis - Treatment<b...,•Do you have a family history of any of the fo...,NaT,NaN,1058,NaN
7,2594,483427,2AL6H-0,F,2016-07-07,8,Suhana Singh Suhana Singh,None,None,None,None,None,2024-01-05 12:49:11,2024-01-05 12:55:20,NaT,nurse1,General Physician,05 Jan 2024 12:55:16,Constipation,►<b>Constipation</b>: <br/>• 4 Years.<br/>• Fr...,None,None,None,5.3000,NaN,NaN,NaN,NaN,NaN,NaN,• Medical History - None.<br/>• Drug history -...,•Do you have a family history of any of the fo.

In [6]:
duration_df

,Host_name,Date_Range,Start,End,Duration
0,LAPTOP-TF6SSSIA,2024-01-01 00:00:00 / 2024-03-31 23:59:59,2024-04-25 15:07:03.762403,2024-04-25 15:07:16.470463,12.70806
